In [2]:
from pathlib import Path
import torch
import torch.nn as nn
from config import get_config, get_weights_file_path
from train import get_model, get_dataset, run_validation

In [3]:
# define the device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Device: {device}")

config = get_config()
train_dataloader, val_dataloader, tokenizer_src, tokenizer_tgt = get_dataset(config)

Device: cpu
Max length of source text: 471
Max length of target text: 482


In [5]:
# Print the first few observations of train_dataloader
for i, batch in enumerate(train_dataloader):
    print(batch)
    if i == 2:  # change this to print more or fewer batches
        break

{'encoder_input': tensor([[   2,  128,   40,  ...,    1,    1,    1],
        [   2,  150,   14,  ...,    1,    1,    1],
        [   2,   13, 1084,  ...,    1,    1,    1],
        ...,
        [   2, 7328, 1351,  ...,    1,    1,    1],
        [   2, 5729,   29,  ...,    1,    1,    1],
        [   2,   46,  474,  ...,    1,    1,    1]]), 'decoder_input': tensor([[   2,  156,  187,  ...,    1,    1,    1],
        [   2,  829,  364,  ...,    1,    1,    1],
        [   2,   34, 7534,  ...,    1,    1,    1],
        ...,
        [   2, 7729,   33,  ...,    1,    1,    1],
        [   2,  384,   46,  ...,    1,    1,    1],
        [   2,  143,  173,  ...,    1,    1,    1]]), 'encoder_mask': tensor([[[[1, 1, 1,  ..., 0, 0, 0]]],


        [[[1, 1, 1,  ..., 0, 0, 0]]],


        [[[1, 1, 1,  ..., 0, 0, 0]]],


        ...,


        [[[1, 1, 1,  ..., 0, 0, 0]]],


        [[[1, 1, 1,  ..., 0, 0, 0]]],


        [[[1, 1, 1,  ..., 0, 0, 0]]]], dtype=torch.int32), 'decoder_mask': tenso

In [10]:
batch.keys()

dict_keys(['encoder_input', 'decoder_input', 'encoder_mask', 'decoder_mask', 'label', 'src_text', 'tgt_text'])

In [58]:
from scratch_transformer import MultiHeadAttentionBlock
from data import create_weights, get_reg_data
import numpy as np

feature_size = 15
output_size = 1
M = 50
N = 10
lr = 1e-17

# linear attention params override
la_params = create_weights(feature_size, output_size, N, lr)

# get the data
eval_data = get_reg_data(no_tasks=M, feature_size=feature_size, no_examples=N)


# Create a MultiHeadAttentionBlock
mha = MultiHeadAttentionBlock(
    d_model=feature_size + 1, heads=1, dropout=0.0, softmax_att=True
)  # (batch_size, seq_len, d_model)

In [59]:
import torch


# Now we will override the weights of the model to implement those that perform GD in the forward pass
def override_weights(model, new_params, w_name):
    w_name = "Transformer_gd/multi_head_attention/" + w_name
    w_numpy = new_params[w_name]["w"]
    w_tensor = torch.tensor(w_numpy, dtype=model.weight.dtype)
    model.weight.data = w_tensor


# Override the weights of the model
override_weights(mha.w_q, la_params, "query")
override_weights(mha.w_k, la_params, "key")
override_weights(mha.w_v, la_params, "value")
override_weights(mha.w_o, la_params, "linear")

In [60]:
def compute_loss(preds, targets):
    """Compute the MSE loss."""
    return 0.5 * np.sum((targets - preds) ** 2) / targets.shape[0]

In [61]:
e = torch.tensor(eval_data[0]).float()

# Forward pass
out = mha(e, e, e)

# Compare the output to the targets
targets = eval_data[1][:, -1]
preds = out[:, -1, -1] * (-1.0)

In [63]:
compute_loss(preds.detach().numpy(), targets)

0.5641194237004725